In [1]:
# MOD 1: needed to append sys path to import holodeck
import sys
sys.path.append('C:/Users/emiga/OneDrive/Cal/GWs/code/holodeck')


# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

Nyquist Frequency Bins/Edges

In [2]:
dur = 5.0*YR/3.1557600
cad = .5*YR/3.1557600
fobs = utils.nyquist_freqs(dur,cad)
fobs_edges = utils.nyquist_freqs_edges(dur,cad)
# print('fobs=', fobs)
# print('fobs_edges=',fobs_edges)
print(f"Number of frequency bins: {fobs.size-1}")
print(f"  between [{fobs[0]*YR:.2f}, {fobs[-1]*YR:.2f}] 1/yr")
print(f"          [{fobs[0]*1e9:.2f}, {fobs[-1]*1e9:.2f}] nHz")

Number of frequency bins: 4
  between [0.63, 3.16] 1/yr
          [20.00, 100.00] nHz


SAM

In [3]:
mtot=(1.0e6*MSOL/1.988409870698051, 1.0e8*MSOL/1.988409870698051, 3)
mrat=(1e-1, 1.0, 2)
redz=(1e-3, 1.0, 4)
sam = holo.sam.Semi_Analytic_Model(mtot=mtot, mrat=mrat, 
        redz=redz)
# gsmf=GSMF_Schechter, 
# gpf=GPF_Power_Law, 
# gmt=GMT_Power_Law, 
# mmbulge=relations.MMBulge_MM2013
   
# print('sam.mtot (M_sun)',sam.mtot/MSOL,
# '\nsam.mtot (g)', sam.mtot,
# '\nsam.mrat',sam.mrat,
# '\nsam.redz',sam.redz)
print('edges:', sam.edges)

edges: [array([1.e+39, 1.e+40, 1.e+41]), array([0.1, 1. ]), array([0.001, 0.01 , 0.1  , 1.   ])]


gwb (internal)

In [4]:
fobs_gw_edges = fobs_edges
hard = holo.hardening.Hard_GW
fobs_gw_cents = kale.utils.midpoints(fobs_gw_edges)
# print('fobs_gw_cents', fobs_gw_cents)
fobs_orb_edges = fobs_gw_edges / 2.0
fobs_orb_cents = fobs_gw_cents / 2.0
# print('fobs_orb_edges', fobs_orb_edges.shape)
# print('fobs_orb_cents', fobs_orb_cents.shape)


# dynamic_binary_number
# gets differential number of binaries per bin-vol
# per log frequency interval
edges, dnum = sam.dynamic_binary_number(hard, fobs_orb=fobs_orb_cents)
edges[-1] = fobs_orb_edges

# integrate (multiply by bin volume) within each
# bin
number = utils._integrate_grid_differential_number(edges, dnum, freq=False)
number = number * np.diff(np.log(fobs_gw_edges))

# get GWB spectrum
hc = holo.sam.gravwaves._gws_from_number_grid_integrated(edges, number, realize=False)
hc1 = holo.sam.gravwaves._gws_from_number_grid_integrated(edges, number, realize=1)
        

In [5]:
# learning
edges_test, dnum_test = sam.dynamic_binary_number(hard, fobs_orb=fobs_orb_cents)
edges_test[-1] = fobs_orb_edges

# integrate (multiply by bin volume) within each
# bin
number1_test = utils._integrate_grid_differential_number(edges_test, dnum_test, freq=False)
number2_test = number1_test * np.diff(np.log(fobs_gw_edges))

print(number1_test)
print('np.diff(log(fobs_gw_edges))\n', np.diff(np.log(fobs_gw_edges)))
print('number2\n', number2_test)

[[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00]
   [8.03509754e+06 1.26544857e+06 4.29207999e+05 1.99295661e+05
    9.65086730e+04]
   [9.95717043e+08 1.56815608e+08 5.31878695e+07 2.46969107e+07
    1.19594479e+07]]]


 [[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00]
   [1.00028124e+05 1.57534424e+04 5.34316736e+03 2.48101171e+03
    1.20142680e+03]
   [1.17415027e+07 1.84917080e+06 6.27191748e+05 2.91226152e+05
    1.41025898e+05]]]]
np.diff(log(fobs_gw_edges))
 [1.09861229 0.51082562 0.33647224 0.25131443 0.28768207]
number2
 [[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00]
   [8.82745690e+06 6.46423553e+05 1.44416576e+05 5.00858751e+04
    2.77638151e+04]
   [1.09390698e+09 8.01054307e+07 1.78962414e+07 6.20668998e+06
    3.44051876e+06]]]


 [[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
    0.00000000e+00]
   [1.09892126e+05 8.04726202e+03 1.79782747e+03 6.235

print our numbers

In [6]:
for num in range(len(fobs_gw_cents)):
    print('\nfobs_gw_cents: %.2eHz' % fobs_gw_cents[num])
    print('fobs_orb_cents: %.2eHz ' % fobs_orb_cents[num])
    print('frequency bin %.2f nHz to %.2f nHz' % (edges[-1][num]*10**9, edges[-1][num+1]*10**9))
    np.set_printoptions(precision = 2)
    print('number =\n', number[:,:,:,num])


fobs_gw_cents: 2.00e-08Hz
fobs_orb_cents: 1.00e-08Hz 
frequency bin 5.00 nHz to 15.00 nHz
number =
 [[[0.00e+00 8.83e+06 1.09e+09]]

 [[0.00e+00 1.10e+05 1.29e+07]]]

fobs_gw_cents: 4.00e-08Hz
fobs_orb_cents: 2.00e-08Hz 
frequency bin 15.00 nHz to 25.00 nHz
number =
 [[[0.00e+00 6.46e+05 8.01e+07]]

 [[0.00e+00 8.05e+03 9.45e+05]]]

fobs_gw_cents: 6.00e-08Hz
fobs_orb_cents: 3.00e-08Hz 
frequency bin 25.00 nHz to 35.00 nHz
number =
 [[[0.00e+00 1.44e+05 1.79e+07]]

 [[0.00e+00 1.80e+03 2.11e+05]]]

fobs_gw_cents: 8.00e-08Hz
fobs_orb_cents: 4.00e-08Hz 
frequency bin 35.00 nHz to 45.00 nHz
number =
 [[[0.00e+00 5.01e+04 6.21e+06]]

 [[0.00e+00 6.24e+02 7.32e+04]]]

fobs_gw_cents: 1.05e-07Hz
fobs_orb_cents: 5.25e-08Hz 
frequency bin 45.00 nHz to 60.00 nHz
number =
 [[[0.00e+00 2.78e+04 3.44e+06]]

 [[0.00e+00 3.46e+02 4.06e+04]]]


# GWs by Loops Dev

In [7]:
foo = edges[-1] # observer frame orbital frequency
print('foo (obs frame orbital freq) =', foo)
df = np.diff(foo)
  # Q? What does np.diff do
  # A! n'th discrete difference along given axis
  #    just the differenc between neighboring values
print('df = diff foo =', df)
fc = kale.utils.midpoints(foo)
print('fc = obs frame orbital freq bin centers =', fc)

foo (obs frame orbital freq) = [5.0e-09 1.5e-08 2.5e-08 3.5e-08 4.5e-08 6.0e-08]
df = diff foo = [1.0e-08 1.0e-08 1.0e-08 1.0e-08 1.5e-08]
fc = obs frame orbital freq bin centers = [1.00e-08 2.00e-08 3.00e-08 4.00e-08 5.25e-08]


calculate GW strain

In [8]:
# midpoints of mtot, mrat, and redz
mt = kale.utils.midpoints(edges[0])
mr = kale.utils.midpoints(edges[1])
rz = kale.utils.midpoints(edges[2])
np.set_printoptions(precision = 5)
print('mt = mtot midpoints =', mt)
print('mr = mrat midpoints =', mr)
print('rz = redz midpoints =',rz)

mt = mtot midpoints = [5.5e+39 5.5e+40]
mr = mrat midpoints = [0.55]
rz = redz midpoints = [0.0055 0.055  0.55  ]


In [9]:
mc = utils.chirp_mass_mtmr(mt[:,np.newaxis], 
                        mr[np.newaxis,:])
mc = mc[:, :, np.newaxis, np.newaxis]
print('mc = chirpmass = \n', mc)
dc = cosmo.comoving_distance(rz).cgs.value
dc = dc[np.newaxis, np.newaxis, :, np.newaxis]
print('dc = comoving distance =\n', dc)


mc = chirpmass = 
 [[[[2.27082e+39]]]


 [[[2.27082e+40]]]]
dc = comoving distance =
 [[[[7.32986e+25]
   [7.25035e+26]
   [6.41542e+27]]]]


In [10]:
# convert from observer frame to rest frame
print('recall fc = obs frame orbital freq bin centers =', fc)
fr = utils.frst_from_fobs(fc[np.newaxis, :], rz[:, np.newaxis])
fr = fr[np.newaxis, np.newaxis, :, :]
print('fr = rest-frame orbital frequency centers =\n',
        fr)

recall fc = obs frame orbital freq bin centers = [1.00e-08 2.00e-08 3.00e-08 4.00e-08 5.25e-08]
fr = rest-frame orbital frequency centers =
 [[[[1.00550e-08 2.01100e-08 3.01650e-08 4.02200e-08 5.27888e-08]
   [1.05500e-08 2.11000e-08 3.16500e-08 4.22000e-08 5.53875e-08]
   [1.55000e-08 3.10000e-08 4.65000e-08 6.20000e-08 8.13750e-08]]]]


strain amplitude of single source (separate for each bin), $h_s$
$$h_s = \frac{A M_c (2M_c f_{r,orb})^{2/3}}{ d_c}$$
where $A$ is a constant,
$$A = \frac{8 \times G^{5/3} \times \pi^{2/3}}{\sqrt(10) c^4}$$

characteristic strain for a single source (separate for each bin)
$$h_c^2 = h_s^2 \times \frac{f_\mathrm{obs,\ orb}}{{df_\mathrm{obs,\ orb}}} 
\\= h_s^2 \times \frac{\mathrm{obs\ frame\ orb\ freq\ center}}{\mathrm{freq\ bin\ width}}$$



* To realize: $h_c^2 = h_c^2 * \mathrm{poisson(number)}$ 
* Multiple realizations: same but with shape thing
* Unrealized: $h_c^2 = h_c^2 * number$ - adds up hc from all the binaries in the bin (not discrete)
\



Number $N = \int \frac{d^3N/dV}{dzd\log{M}dq d\ln{f_r}} d^3N dq dz d\log{M} dV$
If sum, 
$$h_c^2 = \sum_{M,q,z} h_{c,\mathrm{M,q,z}}^2$$

$$h_c = \sqrt{h_c}$$

All together: 
$$
h_c^2 = \sum_{M,q,z} \big(    h_s^2 \times \frac{f_\mathrm{obs,\ orb}}{{df_\mathrm{obs,\ orb}}} \big) \\

\sum_{M,q,z} \big(\frac{A M_c (2M_c f_{r,orb})^{2/3}}{ d_c} \frac{f_\mathrm{obs,\ orb}}{{df_\mathrm{obs,\ orb}}} \big)
$$


In [11]:

# get gw strain from every individual source 
hs = utils.gw_strain_source(mc, dc, fr)
print('hs = single source strain =\n', hs, hs.shape)
print('recall edges =\n', edges)
print('2 total masses x 1 mass ratio x 3 redshifts x 5 frequencies')

hs = single source strain =
 [[[[2.92011e-19 4.63539e-19 6.07407e-19 7.35822e-19 8.82075e-19]
   [3.04824e-20 4.83879e-20 6.34060e-20 7.68109e-20 9.20780e-20]
   [4.45215e-21 7.06734e-21 9.26084e-21 1.12187e-20 1.34486e-20]]]


 [[[1.35540e-17 2.15156e-17 2.81934e-17 3.41538e-17 4.09423e-17]
   [1.41487e-18 2.24597e-18 2.94305e-18 3.56525e-18 4.27388e-18]
   [2.06650e-19 3.28037e-19 4.29850e-19 5.20726e-19 6.24227e-19]]]] (2, 1, 3, 5)
recall edges =
 [array([1.e+39, 1.e+40, 1.e+41]), array([0.1, 1. ]), array([0.001, 0.01 , 0.1  , 1.   ]), array([5.0e-09, 1.5e-08, 2.5e-08, 3.5e-08, 4.5e-08, 6.0e-08])]
2 total masses x 1 mass ratio x 3 redshifts x 5 frequencies


In [12]:
# characteristic strain calculated fromm GW strain
hc = (hs ** 2) * (fc / df)
print('hc = characteristic strain =\n', hc)

hc = characteristic strain =
 [[[[8.52705e-38 4.29736e-37 1.10683e-36 2.16573e-36 2.72320e-36]
   [9.29179e-40 4.68277e-39 1.20610e-38 2.35997e-38 2.96743e-38]
   [1.98216e-41 9.98947e-41 2.57290e-40 5.03438e-40 6.33023e-40]]]


 [[[1.83710e-34 9.25839e-34 2.38460e-33 4.66593e-33 5.86695e-33]
   [2.00186e-36 1.00887e-35 2.59846e-35 5.08440e-35 6.39313e-35]
   [4.27044e-38 2.15217e-37 5.54313e-37 1.08462e-36 1.36381e-36]]]]


hc = (hs ** 2) * (fc / df)

$h_c = h_s^2 \times \frac{\mathrm{obs\ frame\ orb\ freq}}{\mathrm{freq\ bin\ width}}$

# GWs by Loops

In [80]:
def gws_by_loops(edges, number, realize, print_test = False):
       
    """ Inefficient way to calculate strain from numbered 
    grid integrated

    Parameters
    ----------
    edges : (4,) list of 1darrays
        A list containing the edges along each dimension.  The four dimensions correspond to
        total mass, mass ratio, redshift, and observer-frame orbital frequency.
        The length of each of the four arrays is M, Q, Z, F.
    number : (M-1, Q-1, Z-1, F-1) ndarray
        The number of binaries in each bin of parameter space.  This is calculated by integrating
        `dnum` over each bin.
    realize : bool or int,
        Specification of how to construct one or more discrete realizations.
        If a `bool` value, then whether or not to construct a realization.
        If an `int` value, then how many discrete realizations to construct.
    print_test : bool
        Whether or not to print variables as they are calculated, for dev purposes.


    Returns
    -------
    hc : ndarray
        Characteristic strain of the GWB.
        The shape depends on whether realize is an integer or not
        realize = True or False: shape is (M-1, Q-1, Z-1, F-1)
        realize = R: shape is  (M-1, Q-1, Z-1, F-1, R)

    """
    if(print_test):
        print('INPUTS: edges:', len(edges), '\n', edges, 
        '\nINPUTS:number:', number.shape, '\n', number,'\n')

    # Frequency bin midpoints
    foo = edges[-1]                   #: should be observer-frame orbital-frequencies
    df = np.diff(foo)                 #: frequency bin widths
    fc = kale.utils.midpoints(foo)    #: use frequency-bin centers for strain (more accurate!)

    # All other bin midpoints
    mt = kale.utils.midpoints(edges[0]) #: total mass
    mr = kale.utils.midpoints(edges[1]) #: mass ratio
    rz = kale.utils.midpoints(edges[2]) #: redshift

    if(print_test):
        print('Observer frame frequency centers: ', fc)
        print('Mass edges: ', mt.shape, mt)
        print('Ratio edges:', mr.shape, mr)
        print('Redshift edges:', rz.shape, rz,'\n')

    # Chirp mass and comoving distance
    # mc = utils.chirp_mass_mtmr(mt[:, np.newaxis], mr[np.newaxis, :])
    # dc = holo.cosmo.comoving_distance(rz).cgs.value
    # if(print_test):
    #     print('Chirp mass', mc.shape, '\n', mc)
        # print('Comoving distance:', dc,'\n')

    # Convert freq bin centers to rest-frame
    # fr = utils.frst_from_fobs(fc[np.newaxis, :], rz[:, np.newaxis])
    # if(print_test):
    #     print('Rest frame frequency centers:', fr.shape, 
    #         '\n',fr,'\n')
    


    # make strain grid
    hc_grid = np.empty_like(number)

    # for r realizations, make a grid with shape 
    # m, q, z, f, r
    if(utils.isinteger(realize)):
        newshape = hc_grid.shape + (realize,)
        if(print_test):
            print('newshape:', newshape)
        realized_grid = np.empty(newshape)

    # get strain for each bin
    for m_idx in range(len(mt)):
        for q_idx in range(len(mr)):
            for z_idx in range(len(rz)):
                cmass = holo.utils.chirp_mass_mtmr(mt[m_idx], mr[q_idx])
                cdist = holo.cosmo.comoving_distance(rz[z_idx]).cgs.value
                
                # print M, q, z, M_c, d_c
                if(print_test):
                    print('BIN mt=%.2e, mr=%.2e, rz=%.2e' %
                        (mt[m_idx], mr[q_idx], rz[z_idx]))
                    print('\t m_c = %.2e, d_c = %.2e' 
                        % (cmass, cdist))

                for f_idx in range(len(fc)):
                    rfreq = holo.utils.frst_from_fobs(fc[f_idx], rz[z_idx])
                    hs_mqzf = utils.gw_strain_source(cmass, cdist, rfreq)
                    hc_dlnf = hs_mqzf**2 * (fc[f_idx]/df[f_idx])
                    
                    if(realize == False):
                        hc_grid[m_idx, q_idx, z_idx, f_idx] = np.sqrt(hc_dlnf 
                                        * number[m_idx, q_idx, z_idx, f_idx])
                    elif(realize == True):
                        hc_grid[m_idx, q_idx, z_idx, f_idx] = np.sqrt(hc_dlnf 
                                        *np.random.poisson(number[m_idx, q_idx, z_idx, f_idx]))
                    elif(utils.isinteger(realize)):
                        for r_idx in range(realize):
                            realized_grid[m_idx, q_idx, z_idx, f_idx, r_idx] = \
                                np.sqrt(hc_dlnf 
                                        *np.random.poisson(number[m_idx, q_idx, z_idx, f_idx]))
                        hc_grid = realized_grid                
                    else:
                        print("`realize` ({}) must be one of {{True, False, integer}}!"\
                            .format(realize))

                     # print fr, hs, hc^2/dlnf, number, hc
                    if(print_test):
                        print('\t\tfr = %.2fnHz, h_s = %.2e, h_c^2/dlnf = %.2e' 
                            % (rfreq*10**9, hs_mqzf, hc_dlnf))
                        print('\t\tnumber: %.2e' % number[m_idx, q_idx, z_idx, f_idx])
                        print('\t\thc = %.2e' % hc_grid[m_idx, q_idx, z_idx, f_idx])
                    
            

    # if(realize == False):
    #     hc_grid *= number
    # elif(realize == True):
    #     hc_grid *= np.random.poisson(number)

    #not sure if this part will work
    # elif utils.isinteger(realize):
    #     try:
    #         realizations = np.empty(realize)
    #         for real in range(realize): 
    #             realizations[real]= gws_by_loops(edges, number, realize=True)
    #         hc_grid = realizations
    #     except:
    #         print("Something went wrong when you tried to do multiple realizations.")
    print('hc:', hc.shape)

    return hc_grid


    



In [40]:
np.set_printoptions(precision = 4)
loop_hcF = gws_by_loops(edges, number, realize = False, print_test=True)
loop_hcT = gws_by_loops(edges, number, realize = True)
np.set_printoptions(precision = None)

INPUTS: edges: 4 
 [array([1.e+39, 1.e+40, 1.e+41]), array([0.1, 1. ]), array([0.001, 0.01 , 0.1  , 1.   ]), array([5.0e-09, 1.5e-08, 2.5e-08, 3.5e-08, 4.5e-08, 6.0e-08])] 
INPUTS:number: (2, 1, 3, 5) 
 [[[[0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
   [8.8275e+06 6.4642e+05 1.4442e+05 5.0086e+04 2.7764e+04]
   [1.0939e+09 8.0105e+07 1.7896e+07 6.2067e+06 3.4405e+06]]]


 [[[0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
   [1.0989e+05 8.0473e+03 1.7978e+03 6.2351e+02 3.4563e+02]
   [1.2899e+07 9.4460e+05 2.1103e+05 7.3189e+04 4.0571e+04]]]] 

Observer frame frequency centers:  [1.00e-08 2.00e-08 3.00e-08 4.00e-08 5.25e-08]
Mass edges:  (2,) [5.5e+39 5.5e+40]
Ratio edges: (1,) [0.55]
Redshift edges: (3,) [0.0055 0.055  0.55  ] 

BIN mt=5.50e+39, mr=5.50e-01, rz=5.50e-03
	 m_c = 2.27e+39, d_c = 7.33e+25
		fr = 10.06nHz, h_s = 2.92e-19, h_c^2/dlnf = 8.53e-38
		number: 0.00e+00
		hc = 0.00e+00
		fr = 20.11nHz, h_s = 4.64e-19, h_c^2/dlnf = 4.30e-37
		number: 0.00e+00


In [82]:
def gws_by_arrs(edges, number, realize, sum=False, print_test=False):


    foo = edges[-1]                   #: should be observer-frame orbital-frequencies
    df = np.diff(foo)                 #: frequency bin widths
    fc = kale.utils.midpoints(foo)    #: use frequency-bin centers for strain (more accurate!)

    # ---- calculate GW strain ----
    mt = kale.utils.midpoints(edges[0])
    mr = kale.utils.midpoints(edges[1])
    rz = kale.utils.midpoints(edges[2])
    mc = utils.chirp_mass_mtmr(mt[:, np.newaxis], mr[np.newaxis, :])
    mc = mc[:, :, np.newaxis, np.newaxis]
    if(print_test):
        print('chirp mass: ', mc.shape, '\n', mc)
        print(mc[0,0,0,0])
        print(mc[1,0,0,0])
    dc = cosmo.comoving_distance(rz).cgs.value
    dc = dc[np.newaxis, np.newaxis, :, np.newaxis]

    if(print_test):
        print('Observer frame frequency centers: ', fc)
        print('Mass edges: ', mt.shape, mt)
        print('Ratio edges:', mr.shape, mr)
        print('Redshift edges:', rz.shape, rz,'\n')

    

    # convert from observer-frame to rest-frame; still using frequency-bin centers
    fr = utils.frst_from_fobs(fc[np.newaxis, :], rz[:, np.newaxis])
    fr = fr[np.newaxis, np.newaxis, :, :]

    hs = utils.gw_strain_source(mc, dc, fr)
    hc = (hs ** 2) * (fc / df)
    print('hs:', hs.shape,'\n',hs)
    print('hc^2/dlnf:', hc.shape,'\n',hc)
    # print('mt', len(mt))
    # print('dc', len(dc), dc.shape)
    # print chirp mass and comoving distance
    if(print_test):
        for m_idx in range(len(mt)):
            for q_idx in range(len(mr)):
                for z_idx in range(len(rz)):
                    print('BIN mt=%.2e, mr=%.2e, rz=%.2e' %
                        (mt[m_idx], mr[q_idx], rz[z_idx]))
                    # print('m_idx = %d, q_idx = %d, z_idx=%d' % (m_idx, q_idx, z_idx))
                    print('\t m_c = %.2e, d_c = %.2e' 
                        % (mc[m_idx, q_idx, 0,0], dc[0, 0, z_idx,0]))
                    for f_idx in range(len(fc)):
                        print('\t\tfr = %.2fnHz, h_s = %.2e, h_c^2/dlnf = %.2e' 
                            % (fr[0, 0, 0, f_idx]*10**9, 
                            hs[m_idx, q_idx, z_idx, f_idx], hc[m_idx, q_idx, z_idx, f_idx]))  
                        print('\t\tnumber: %.2e' % number[m_idx, q_idx, z_idx, f_idx])
    # if(print_test):
    #     for m_idx in range(len(mt)):
    #         for q_idx in range(len(mr)):
    #             for z_idx in range(len(rz)):


    if realize is True:
        hc = hc * np.random.poisson(number)
    elif realize in [None, False]:
        # print('err here')
        # print('hc:\n', hc, '\nhc shape:', hc.shape)
        # print('number:\n', number, '\nnumber shape', number.shape)
        hc = hc * number
    elif utils.isinteger(realize):
        shape = number.shape + (realize,)
        try:
            hc = hc[..., np.newaxis] * np.random.poisson(number[..., np.newaxis], size=shape)
        except ValueError as err:
            log.error(str(err))
            print(f"{utils.stats(number)=}")
            print(f"{number.max()=:.8e}")
            print(f"{number.dtype=}")
            raise
    else:
        err = "`realize` ({}) must be one of {{True, False, integer}}!".format(realize)
        log.error(err)
        raise ValueError(err)

    # Sum over M, Q, Z bins
    # (M-1, Q-1, Z-1, F-1 [, R]) ==> (F-1, [, R])
    if sum:
        hc = np.sum(hc, axis=(0, 1, 2))

    # convert from hc^2 to hc
    hc = np.sqrt(hc)
    print('hc shape:', hc.shape)
    return hc

In [32]:
np.set_printoptions(precision = 4)
print('\n-----------by loops--------')
hc_loopF = gws_by_loops(edges, number, realize = False, print_test=True)
print('\n-----------by grid arrays--------')
hc_arrF = gws_by_arrs(edges, number, realize = False, print_test=True)

# hc_arrT = gws_by_arrs(edgeas, number, realize = True)
np.set_printoptions(precision = None)


-----------by loops--------
INPUTS: edges: 4 
 [array([1.e+39, 1.e+40, 1.e+41]), array([0.1, 1. ]), array([0.001, 0.01 , 0.1  , 1.   ]), array([5.0e-09, 1.5e-08, 2.5e-08, 3.5e-08, 4.5e-08, 6.0e-08])] 
INPUTS:number: (2, 1, 3, 5) 
 [[[[0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
   [8.8275e+06 6.4642e+05 1.4442e+05 5.0086e+04 2.7764e+04]
   [1.0939e+09 8.0105e+07 1.7896e+07 6.2067e+06 3.4405e+06]]]


 [[[0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
   [1.0989e+05 8.0473e+03 1.7978e+03 6.2351e+02 3.4563e+02]
   [1.2899e+07 9.4460e+05 2.1103e+05 7.3189e+04 4.0571e+04]]]] 

Observer frame frequency centers:  [1.00e-08 2.00e-08 3.00e-08 4.00e-08 5.25e-08]
Mass edges:  (2,) [5.5e+39 5.5e+40]
Ratio edges: (1,) [0.55]
Redshift edges: (3,) [0.0055 0.055  0.55  ] 

BIN mt=5.50e+39, mr=5.50e-01, rz=5.50e-03
	 m_c = 2.27e+39, d_c = 7.33e+25
		fr = 10.06nHz, h_s = 2.92e-19, h_c^2/dlnf = 8.53e-38
		number: 0.00e+00
		fr = 20.11nHz, h_s = 4.64e-19, h_c^2/dlnf = 4.30e-37
		numb

In [ ]:
# print(arr_hcF)
# print(loop_hcF)
print(hc_samF == hc_arrF)

[[[[ True  True  True  True  True]
   [ True  True  True  True  True]
   [ True  True  True  True  True]]]


 [[[ True  True  True  True  True]
   [ True  True  True  True  True]
   [ True  True  True  True  True]]]]


# Test Cases

In [85]:
# get GWB spectrum
hc_samF = holo.sam.gravwaves._gws_from_number_grid_integrated(edges, number, realize=False, sum=False)
hc_samT = holo.sam.gravwaves._gws_from_number_grid_integrated(edges, number, realize=True, sum=False)
hc_sam1 = holo.sam.gravwaves._gws_from_number_grid_integrated(edges, number, realize=1, sum=False)
hc_sam3 = holo.sam.gravwaves._gws_from_number_grid_integrated(edges, number, realize=3, sum=False)

hc_loopF = gws_by_loops(edges, number, realize=False)    
hc_loopT = gws_by_loops(edges, number, realize=True)
hc_loop1 = gws_by_loops(edges, number, realize=1)   
hc_loop3 = gws_by_loops(edges, number, realize=3)   
        

hc: (2, 1, 3, 5)
hc: (2, 1, 3, 5)
hc: (2, 1, 3, 5)
hc: (2, 1, 3, 5)


In [86]:
print(hc_loop3.shape)

(2, 1, 3, 5, 3)


In [87]:
print(hc_sam3.shape)

(2, 1, 3, 5, 3)


In [88]:
print(hc_loop3 - hc_sam3)

[[[[[ 0.0000e+00  0.0000e+00  0.0000e+00]
    [ 0.0000e+00  0.0000e+00  0.0000e+00]
    [ 0.0000e+00  0.0000e+00  0.0000e+00]
    [ 0.0000e+00  0.0000e+00  0.0000e+00]
    [ 0.0000e+00  0.0000e+00  0.0000e+00]]

   [[-1.9683e-20  2.6558e-20  8.8436e-21]
    [-2.1019e-20 -8.5495e-20 -1.7012e-20]
    [ 1.4265e-20  1.7781e-20 -1.4135e-19]
    [ 3.9402e-20 -1.0746e-19  9.7715e-20]
    [ 1.6062e-20 -7.6559e-20 -5.5678e-20]]

   [[-3.7782e-21  3.4444e-21 -3.5882e-21]
    [-2.1570e-21 -6.2673e-21 -1.5290e-20]
    [-2.2537e-20 -7.2036e-21  5.5436e-21]
    [-1.1255e-22 -1.0019e-20 -2.6469e-20]
    [ 9.6761e-21  1.8102e-20 -2.9819e-20]]]]



 [[[[ 0.0000e+00  0.0000e+00  0.0000e+00]
    [ 0.0000e+00  0.0000e+00  0.0000e+00]
    [ 0.0000e+00  0.0000e+00  0.0000e+00]
    [ 0.0000e+00  0.0000e+00  0.0000e+00]
    [ 0.0000e+00  0.0000e+00  0.0000e+00]]

   [[-8.1462e-19  1.9412e-19 -8.1094e-20]
    [-8.1363e-19 -3.1030e-18 -2.3358e-18]
    [-2.3671e-18  7.2780e-19  0.0000e+00]
    [ 1.2871e-18  2.14

In [ ]:
print(hc_loopF)

[[[[8.5270e-38 4.2974e-37 1.1068e-36 2.1657e-36 2.7232e-36]
   [9.2918e-40 4.6828e-39 1.2061e-38 2.3600e-38 2.9674e-38]
   [1.9822e-41 9.9895e-41 2.5729e-40 5.0344e-40 6.3302e-40]]]


 [[[1.8371e-34 9.2584e-34 2.3846e-33 4.6659e-33 5.8669e-33]
   [2.0019e-36 1.0089e-35 2.5985e-35 5.0844e-35 6.3931e-35]
   [4.2704e-38 2.1522e-37 5.5431e-37 1.0846e-36 1.3638e-36]]]]
